# Bygga AI-agenter med bestående minne med Cognee

Den här notebooken visar hur man bygger intelligenta AI-agenter med avancerade minnesfunktioner med hjälp av [**cognee**](https://www.cognee.ai/) - ett öppet AI-minne som kombinerar kunskapsgrafer, semantisk sökning och sessionshantering för att skapa kontextmedvetna AI-system.

## 🎯 Lärandemål

I slutet av denna handledning kommer du att förstå hur man:
- **Bygger kunskapsgrafer baserade på embeddings**: Omvandlar ostrukturerad text till strukturerad, sökbar kunskap
- **Implementerar sessionsminne**: Skapar flervändiga konversationer med automatisk kontextbevarande
- **Sparar konversationer**: Valfritt lagrar viktiga interaktioner i långtidsminne för framtida referens
- **Frågar med naturligt språk**: Åtkomst till och utnyttjande av historisk kontext i nya konversationer
- **Visualiserar minne**: Utforskar relationerna i agentens kunskapsgraf


## 🏗️ Vad du kommer att bygga

I den här handledningen kommer vi att skapa en **Kodningsassistent** med ihållande minne som:

### 1. **Kunskapsbasuppbyggnad**
   - Tar in information om utvecklarprofiler och expertis
   - Bearbetar principer och bästa praxis för Python-programmering
   - Lagrar historiska konversationer mellan utvecklare och AI-assistenter

### 2. **Sessionsmedvetna konversationer**
   - Bibehåller kontext över flera frågor i samma session
   - Cachar automatiskt varje fråga/svar-par för effektiv hämtning
   - Ger sammanhängande och kontextuella svar baserade på konversationshistorik

### 3. **Långtidsminne**
   - Sparar viktiga konversationer i ett långtidsminne
   - Hämtar relevanta minnen från kunskapsbasen och tidigare sessioner för att informera nya interaktioner
   - Bygger en växande kunskapsbas som förbättras över tid

### 4. **Intelligent minneshämtning**
   - Använder grafmedveten semantisk sökning för att hitta relevant information i all lagrad kunskap
   - Filtrerar sökningar efter datasubgrupper (utvecklarinformation vs. principer)
   - Kombinerar flera datakällor för att ge omfattande svar


## 📋 Förutsättningar & Installation

### Systemkrav

Innan du börjar, se till att du har:

1. **Python-miljö**
   - Python 3.9 eller högre
   - Virtuell miljö (rekommenderas)
   
2. **Redis Cache** (Krävs för sessionshantering)
   - Lokal Redis: `docker run -d -p 6379:6379 redis`
   - Eller använd en hanterad Redis-tjänst
   
3. **LLM API-åtkomst**
   - OpenAI API-nyckel eller andra leverantörer (se [dokumentation](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Databaskonfiguration**
   - Ingen konfiguration krävs som standard. Cognee använder filbaserade databaser (LanceDB och Kuzu)
   - Alternativt kan du konfigurera Azure AI Search som en vektorlagring (se [dokumentation](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Miljökonfiguration

Skapa en `.env`-fil i din projektkatalog med följande variabler:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Förstå Cognees minnesarkitektur

### Hur Cognee fungerar

Cognee erbjuder ett avancerat minnessystem som går bortom enkel nyckel-värde-lagring:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Viktiga komponenter:

1. **Kunskapsgraf**: Lagrar enheter, relationer och semantiska kopplingar
2. **Vektorinbäddningar**: Möjliggör semantisk sökning över all lagrad information
3. **Sessionscache**: Bibehåller samtalskontext inom och mellan sessioner
4. **NodeSets**: Organiserar data i logiska kategorier för riktad hämtning

### Minnestyper i denna handledning:

- **Persistent Memory**: Långtidslagring i kunskapsgrafen
- **Session Memory**: Tillfällig samtalskontext i Redis-cache
- **Semantic Memory**: Vektorbaserad likhetssökning över all data


## 📦 Installera nödvändiga paket

Installera Cognee med Redis-stöd för sessionshantering:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Initiera miljö och ladda bibliotek

Se till att:
1. Redis körs (t.ex. via Docker: `docker run -d -p 6379:6379 redis`)
2. Miljövariabler är inställda innan cachemoduler importeras
3. Om det behövs, starta om kärnan och kör cellerna i ordning

Följande cell kommer att:
1. Ladda miljövariabler från `.env`
2. Konfigurera Cognee med dina LLM-inställningar
3. Aktivera caching för sessionshantering
4. Kontrollera att alla komponenter är korrekt anslutna


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Konfigurera lagringskataloger

Cognee använder två separata kataloger för sina operationer:
- **Data Root**: Lagrar importerade dokument och bearbetad data
- **System Root**: Innehåller kunskapsgrafdatabasen och systemmetadata

Vi kommer att skapa separata kataloger för denna handledning enligt följande:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Återställ minnestillstånd

Innan vi börjar bygga vårt minnssystem, låt oss se till att vi börjar från början.

> 💡 **Tips**: Du kan hoppa över detta steg om du vill behålla befintliga minnen från dina tidigare körningar när du använder denna notebook senare.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Del 1: Bygga kunskapsbasen

### Datakällor för vår utvecklarassistent

Vi kommer att använda tre typer av data för att skapa en omfattande kunskapsbas:

1. **Utvecklarprofil**: Personlig expertis och teknisk bakgrund
2. **Python bästa praxis**: The Zen of Python med praktiska riktlinjer
3. **Historiska konversationer**: Tidigare frågor och svar mellan utvecklare och AI-assistenter

Denna mångsidiga data gör det möjligt för vår agent att:
- Förstå användarens tekniska sammanhang
- Tillämpa bästa praxis i rekommendationer
- Lära sig från tidigare framgångsrika interaktioner


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Bearbeta data till kunskapsgraf

Nu ska vi omvandla vår råtext till ett strukturerat minne. Denna process:

1. **Lägger till data i NodeSets**: Organiserar information i logiska kategorier
   - `developer_data`: Utvecklarprofil och konversationer
   - `principles_data`: Bästa praxis och riktlinjer för Python

2. **Kör Cognify Pipeline**: Extraherar entiteter, relationer och skapar embeddings
   - Identifierar nyckelkoncept
   - Skapar semantiska kopplingar mellan relaterad information
   - Genererar vektorembeddings

Detta kan ta några ögonblick medan LLM bearbetar texten och bygger grafstrukturen:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Visualisera kunskapsgrafen

Låt oss utforska strukturen av vår kunskapsgraf. Visualiseringen visar:
- **Noder**: Enheter som extraherats från texten (koncept, teknologier, personer)
- **Kanter**: Relationer och kopplingar mellan enheter
- **Kluster**: Relaterade koncept grupperade efter semantisk likhet

Öppna den genererade HTML-filen i din webbläsare för att interaktivt utforska grafen:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Berika minnet med Memify

Funktionen `memify()` analyserar kunskapsgrafen och genererar intelligenta regler om datan. Denna process:
- Identifierar mönster och bästa praxis
- Skapar handlingsbara riktlinjer baserade på innehållet
- Etablerar relationer mellan olika kunskapsområden

Dessa regler hjälper agenten att fatta mer informerade beslut när den svarar på frågor. Att fånga en andra visualisering hjälper dig att jämföra hur grafen blir tätare när den berikas.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Del 2: Intelligent informationshämtning

### Demonstration 1: Kunskapsintegration över dokument

Nu när vår kunskapsgraf är byggd, låt oss testa hur Cognee kombinerar information från flera källor för att besvara komplexa frågor.

Den första frågan demonstrerar:
- **Semantisk förståelse**: Hitta relevanta koncept även när de inte uttryckligen nämns
- **Korsreferenser**: Kombinera utvecklarprofil med Python-principer
- **Kontextuell resonemang**: Tillämpa bästa praxis på specifika implementationer

### Demonstration 2: Filtrerad sökning med NodeSets

Den andra frågan visar hur man riktar in sig på specifika delmängder av kunskapsgrafen:
- Använder `node_name`-parametern för att söka endast inom `principles_data`
- Ger fokuserade svar från ett specifikt kunskapsområde
- Användbart när du behöver information från ett specifikt område


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Del 3: Inställning av sessionshantering

### Aktivera konversationsminne

Sessionshantering är avgörande för att behålla kontexten över flera interaktioner. Här kommer vi att:

1. **Initiera användarkontext**: Skapa eller hämta en användarprofil för att spåra sessioner
2. **Konfigurera cachemotor**: Anslut till Redis för att lagra konversationshistorik
3. **Aktivera sessionsvariabler**: Ställ in kontextvariabler som kvarstår mellan frågor

> ⚠️ **Viktigt**: Detta kräver att Redis körs och att `CACHING=true` är inställt i din miljö


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Hjälpfunktion: Visa sessionshistorik

Den här hjälpfunktionen gör det möjligt för oss att granska konversationshistoriken som lagras i Redis. Den är användbar för:
- Felsökning av sessionshantering
- Verifiera att konversationer cachas
- Förstå vilken kontext som är tillgänglig för agenten


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Session 1: Async Support Lab — Första Frågan

Start `async-support-lab`-sessionen genom att fråga efter telemetrivänliga asyncio-mönster för en massiv webbscraper. Grafen känner redan till asyncio, aiohttp och övervakningspraxis, så svaret bör spegla tidigare samtal samtidigt som det anpassas till den nya frågan.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Inspektera minnet för Session 1 efter det första utbytet

Att köra `show_history(session_1)` direkt efter den inledande frågan bekräftar att Cognee skrev både prompten och svaret till Redis. Du bör se en post med vägledning för samtidighet.


In [ ]:
await show_history(session_1)

## Session 1: Uppföljning på Datamodeller

Nästa fråga är: "När ska jag välja dataclasses istället för Pydantic?" med samma sessions-ID. Cognee bör sammanfoga Python-principerna med tidigare FastAPI-diskussioner för att ge nyanserade råd—och visa att kontexten följer med inom en namngiven session.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Bekräfta att historiken för session 1 innehåller båda turerna

Ett ytterligare anrop av `show_history(session_1)` bör visa två frågor och svar. Detta stämmer överens med Mem0-labbets steg för "minnesuppspelning" och bevisar att ytterligare turer läggs till i samma transkript.


In [ ]:
await show_history(session_1)

## Session 2: Designgranskningstråd — Ny session

För att visa isolering mellan trådar startar vi `design-review-session` och ber om loggningsvägledning för incidentgranskningar. Även om den underliggande kunskapsbasen är densamma, håller det nya sessions-ID:t transkriptionerna separata.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Granska Session 2 Historia

`show_history(session_2)` ska endast lista designgranskningens prompt/svar-par. Jämför detta med Session 1 för att belysa hur Cognee håller separata transkript samtidigt som den återanvänder den gemensamma kunskapsgrafen.


In [ ]:
await show_history(session_2)

## Sammanfattning

Grattis! Du har precis gett din kodassistent ett verkligt långtidsminneslager drivet av Cognee.

I den här handledningen tog du rått utvecklarinnehåll (kod, dokumentation, chattar) och omvandlade det till ett graf- och vektorminne som din agent kan söka i, resonera kring och kontinuerligt förbättra.

Vad du har lärt dig

1. **Från råtext till AI-minne**: Hur Cognee tar in ostrukturerad data och omvandlar den till ett intelligent, sökbart minne med hjälp av en kombinerad vektor- och kunskapsgrafarkitektur.

2. **Grafberikning med memify**: Hur du kan gå bortom grundläggande grafskapande och använda memify för att lägga till härledda fakta och rikare relationer ovanpå din befintliga graf.

3. **Flera sökstrategier**: Hur du kan söka i minnet med olika söktyper (grafmedveten Q&A, RAG-stil komplettering, insikter, råa textbitar, kodsökning, etc.) beroende på vad din agent behöver.

4. **Visuell utforskning**: Hur du kan inspektera och felsöka det som Cognee har byggt med hjälp av grafvisualiseringar och Cognee UI, så att du faktiskt kan se hur kunskapen är strukturerad.

5. **Sessionsmedvetet minne**: Hur du kombinerar kontext per session med ett bestående semantiskt minne så att agenter kan komma ihåg mellan körningar utan att läcka information mellan användare.


## Viktiga insikter
1. Minne som en kunskapsgraf stödd av inbäddningar

    - **Strukturerad förståelse**: Cognee kombinerar en vektorlagring och en graflagring så att din data både är sökbar efter betydelse och kopplad genom relationer. Cognee använder filbaserade databaser som standard (LanceDB för vektor-, Kuzu för graflagring).

    - **Relationsmedveten hämtning**: Svar kan baseras inte bara på "liknande text" utan också på hur entiteter är relaterade.

    - **Levande minne**: Minneslagret utvecklas, växer och förblir sökbart som en sammanhängande graf.

2. Sök- och resonemangslägen
    - **Hybridhämtning**: Sökningen kombinerar vektorsimilaritet, grafstruktur och LLM-resonemang, från rå chunk-uppslag till grafmedveten frågesvar.

    - **Anpassa läget till uppgiften**: Använd lägen för textkomplettering när du vill ha naturliga språksvar, och chunk-/sammanfattnings-/graflägen när din agent behöver rå kontext eller för att driva sitt eget resonemang.

3. Personliga, sessionsmedvetna agenter
    - **Sessionskontext + långtidsminne**: Cognee håller kortsiktig "tråd"-kontext separat från långlivat, användar- eller organisationsnivåminne.

## Verkliga tillämpningar

1. **Vertikala AI-agenter**

    Använd mönstret från denna notebook för att driva domänsmarta copiloter som bygger på Cognee som sin kärna för hämtning och resonemang:

- **Utvecklarcopiloter**: Kodgranskning, incidentanalys och arkitekturassistenter som navigerar genom kod, API:er, design-dokument och ärenden som en enda minnesgraf.

- **Kundfokuserade copiloter**: Support- eller framgångsagenter som hämtar från produktdokumentation, vanliga frågor, CRM-anteckningar och tidigare ärenden med grafmedveten hämtning och citerade svar.

- **Interna expertcopiloter**: Policy-, juridik- eller säkerhetsassistenter som resonerar över sammanhängande regler, riktlinjer och historiska beslut istället för isolerade PDF-filer.

    Cognee är uttryckligen positionerat som ett bestående, exakt minne för AI-agenter, och tillhandahåller en levande kunskapsgraf som integreras bakom din agent och ersätter ad-hoc-kombinationer av vektorlager och anpassad grafkod.

2. **Sammanföra datasilos till ett minne**

    Samma tillvägagångssätt hjälper dig också att bygga ett enhetligt minneslager över spridda källor:

- **Från silos till en graf**: Importera strukturerad (t.ex. databaser) och ostrukturerad data (t.ex. dokument, chattar) till en enda graf stödd av inbäddningar, istället för separata index för varje system.

- **Källöverskridande resonemang med citat**: Kör flerstegsresonemang över allt—"koppla samman" loggar, mätvärden och dokument via grafen—och returnera ändå grundade svar med proveniens.

- **Kunskapshubbar**: För områden som bank eller utbildning används Cognee redan för att förena PDF-filer, interna system och appdata till en kunskapsgraf med vektorer så att agenter kan besvara frågor med exakt, citerad kontext.

## Nästa steg

Du har implementerat den grundläggande minnesloopen. Här är naturliga utvidgningar du kan prova själv (se [Cognee-dokumentation](https://docs.cognee.ai/) för detaljer):

1. **Experimentera med tidsmedvetenhet**: Aktivera temporal cognify för att extrahera händelser och tidsstämplar från text.

2. **Introducera ontologidrivet resonemang**: Definiera en OWL-ontologi för ditt område. Använd Cognees ontologistöd så att extraherade entiteter och relationer är förankrade i det schemat, vilket förbättrar grafkvaliteten och domänspecifika svar.

3. **Lägg till en feedback-loop**: Låt Cognee justera grafkantvikter baserat på verklig användarfeedback, så att hämtningen förbättras över tid istället för att förbli statisk.

4. **Optimera för personalisering och sessionsbeteende**: Använd användar-ID:n, hyresgäster och dataset för att ge varje person eller team sin egen vy över den delade minnesmotorn.

5. **Skala upp till mer komplexa agenter**: Anslut Cognee till agentramverk för att bygga multi-agent-system som alla delar samma minneslager. *Microsoft Agent Framework x Cognee-plugin kommer snart.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör det noteras att automatiserade översättningar kan innehålla fel eller felaktigheter. Det ursprungliga dokumentet på dess ursprungliga språk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
